In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.2. Biomarker Distribution Analysis/Output')
cur_date = "050423"

library(readxl)
library(tidyverse)
library(reshape2)
library(stats)
library(multcomp)

# reading in files
mRNA_df = data.frame(read_excel("Input/Imputed_mRNA_Data_042623.xlsx"))
demographics_df = data.frame(read_excel("Input/Subject_Info_031723.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: mvtnorm

Loading required package: survival

Loading required package: TH.data

Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select



Attaching package: ‘TH.data’


The following object is masked from ‘package:MASS’:

    geyser




In [2]:
head(mRNA_df)
head(demographics_df)

,Subject_ID,Condensate,Burn_Condition,Concentration,Time_Point,mRNA,ddCT_pslog2
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,M_6,PBS,PBS,NA,24,HMOX1,4.871886
2,M_6,PBS,PBS,NA,24,ALDH3A1,4.837012
3,M_6,PBS,PBS,NA,24,CXCL1,4.822372
4,M_6,PBS,PBS,NA,24,CXCR1,3.940873
5,M_6,PBS,PBS,NA,24,GCLC,4.863186
6,M_6,PBS,PBS,NA,24,GCLM,4.834127


,Original_Subject_ID,Subject_ID,Subject_No,Sex,Age,Race,Ethnicity
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
1,68J,F_1,1,F,18,B,NH
2,53K,M_2,2,M,32,B,NH
3,21O,M_3,3,M,34,W,H
4,63O,F_4,4,F,32,W,H
5,67L,F_5,5,F,17,W,H
6,57N,M_6,6,M,59,W,NH


Testing for statistical differences by comparing a mRNA expression between burn conditions (control, smoldering or flaming). (The condensate, concentration, and time point would remain consistent). 

This time sex will be added to models as a covariate. I couldn't find a non-parametric test in R that would allow for the addition of a covariate, therefore crude ANOVA models/ Tukey's post hoc test will be run first and then ANCOVA models/ Tukey's post hoc test that include sex.

In [3]:
#  combining data, but first making sex a factor
demographics_df = demographics_df %>%
    mutate(Sex = relevel(factor(ifelse(Sex == "M", 1, 0)), ref = "0"))

mRNA_demographics_df = inner_join(mRNA_df, demographics_df[,c(2,4)]) %>%
    # scaling the concentration values by mRNA normalizes the distribution
    group_by(mRNA) %>%
    mutate(Scaled_Intensity = scale(ddCT_pslog2))

# putting burn condition into a factor
mRNA_demographics_df$Burn_Condition = factor(mRNA_demographics_df$Burn_Condition, levels = c("PBS", "S", "F"))

Joining, by = "Subject_ID"


In [4]:
# the mRNA df doesn't consistently test all 3 concentrations (1,5, or 25 micrograms) or 
# time points(4,24,72) therefore we can't subset the df using a loop so we'll create separate dfs
# for each concentration and time point
split_mRNA_df = mRNA_demographics_df %>%
    group_by(Concentration, Time_Point) %>%
    group_split()

conc1_24_mRNA_df = split_mRNA_df[[1]]
conc1_72_mRNA_df = split_mRNA_df[[2]]
conc25_4_mRNA_df = split_mRNA_df[[3]]
conc25_24_mRNA_df = split_mRNA_df[[4]]
conc25_72_mRNA_df = split_mRNA_df[[5]]
conc5_24_mRNA_df = split_mRNA_df[[6]]
conc5_72_mRNA_df = split_mRNA_df[[7]]
concc_4_mRNA_df = split_mRNA_df[[8]]
concc_24_mRNA_df = split_mRNA_df[[9]]
concc_72_mRNA_df = split_mRNA_df[[10]]

In [5]:
# control samples have a concentration of NA, so they're all in a separate df
# adding them back into the other dataframes
conc1_24_mRNA_df = unique(rbind(conc1_24_mRNA_df, concc_24_mRNA_df))
conc1_72_mRNA_df = unique(rbind(conc1_72_mRNA_df, concc_72_mRNA_df))
conc25_4_mRNA_df = unique(rbind(conc25_4_mRNA_df, concc_4_mRNA_df))
conc25_24_mRNA_df = unique(rbind(conc25_24_mRNA_df, concc_24_mRNA_df))
conc25_72_mRNA_df = unique(rbind(conc25_72_mRNA_df, concc_72_mRNA_df))
conc5_24_mRNA_df = unique(rbind(conc5_24_mRNA_df, concc_24_mRNA_df))
conc5_72_mRNA_df = unique(rbind(conc5_72_mRNA_df, concc_72_mRNA_df))

head(conc1_24_mRNA_df)

Subject_ID,Condensate,Burn_Condition,Concentration,Time_Point,mRNA,ddCT_pslog2,Sex,Scaled_Intensity
<chr>,<chr>,<fct>,<chr>,<dbl>,<chr>,<dbl>,<fct>,"<dbl[,1]>"
M_6,P,F,1,24,HMOX1,4.774776,1,-1.1930586
M_6,P,F,1,24,ALDH3A1,4.762372,1,-1.3544823
M_6,P,F,1,24,CXCL1,4.768515,1,-1.2079038
M_6,P,F,1,24,CXCR1,3.814665,1,-0.5503481
M_6,P,F,1,24,GCLC,4.792038,1,-1.4014199
M_6,P,F,1,24,GCLM,4.814864,1,-1.3361943


In [6]:
# contrasts show what compartments are being compared in anova
# these comparisons aren't what we want so I changed them in the function below
contrasts(mRNA_demographics_df$Burn_Condition)

,S,F
PBS,0,0
S,1,0
F,0,1


  Table of Contrasts 
>               PBS |  S  |  F  |  Sum
>  - Contrast 1 |  -1  |  1  |  0  |   0
>  - Contrast 2 |  -1  |  0  |  1  |   0

> - Contrast 1: compares PBS to smoldering
> - Contrast 2: compares PBS to flaming

In [7]:
get_anova = function(df){
    # """
    # Running anova/ tukey's tests after filtering for gene and condensate using a loop. 
    # Ultimately using this test to compare gene expression (control vs. flaming vs. smoldering burn condition).

    # :param: subsetted dataframe
    # :output: a dataframe containing the gene, condensate, comparison, conc, time point, stat, p value, p adj

    # """
    
    # first filtering the df and iterating through the variables
    genes = unique(df$mRNA)
    condensates = c("C", "P")
    conc = unique(df$Concentration)[1]
    time = unique(df$Time_Point)[1]
    
    tukey_df = data.frame()
    for(i in 1:length(genes)){
        for(j in 1:length(condensates)){
            filtered_df = df %>%
                filter(mRNA == genes[i], Condensate %in% c(condensates[j], "PBS"))

            # changing contrasts
            contrasts(filtered_df$Burn_Condition) = cbind(c(-1,1,0), c(-1,0,1)) # meaning is specified above
            # ANOVA
            anova = aov(Scaled_Intensity ~ Burn_Condition, data = filtered_df)

            # Tukey's post hoc test
            tukeys_anova = summary(glht(anova, linfct = mcp(Burn_Condition = "Tukey")), test = adjusted("none"))
            tukeys_test = tukeys_anova$test
            values_df = data.frame("ANOVA", condensates[j], genes[i], conc, time, tukeys_test$tstat, 
                                   tukeys_test$pvalues) %>%
                rownames_to_column(var = "Comparison")

            # saving values
            tukey_df = rbind(tukey_df, values_df)
        }
    }
    
    # changing col names
    colnames(tukey_df)[2:8] = c("Model", "Condensate", "mRNA", "Concentration", "Time Point", "Statistic", 
                                "P Value")
    
 
    # calculating padj values
    PAdj = c()
    for(j in 1:length(condensates)){
        filtered_df = tukey_df %>%
            filter(Condensate == condensates[j])
        padj = p.adjust(as.numeric(as.character(filtered_df$`P Value`)), method = "fdr")
        PAdj = c(PAdj, padj)

    }
    
    tukey_df$`P Adj` = PAdj
    
    return(tukey_df)
}

# calling fn
mRNA_conc1_24_anova_tukey = get_anova(conc1_24_mRNA_df)
mRNA_conc1_72_anova_tukey = get_anova(conc1_72_mRNA_df)
mRNA_conc25_4_anova_tukey = get_anova(conc25_4_mRNA_df)
mRNA_conc25_24_anova_tukey = get_anova(conc25_24_mRNA_df)
mRNA_conc25_72_anova_tukey = get_anova(conc25_72_mRNA_df)
mRNA_conc5_24_anova_tukey = get_anova(conc5_24_mRNA_df)
mRNA_conc5_72_anova_tukey = get_anova(conc5_72_mRNA_df)

In [8]:
get_ancova = function(df){
    # """
    # Running anova/ tukey's tests after filtering for gene and condensate using a loop. 
    # Ultimately using this test to compare gene expression (control vs. smoldering vs. flaming burn condition).
    # Adding in sex as a covariate.

    # :param: subsetted dataframe
    # :output: a dataframe containing the gene, condensate, comparison, conc, time point, stat, p value, p adj

    # """
    
   # first filtering the df and iterating through the variables
    genes = unique(df$mRNA)
    condensates = c("C", "P")
    conc = unique(df$Concentration)[1]
    time = unique(df$Time_Point)[1]
    
    tukey_df = data.frame()
    for(i in 1:length(genes)){
        for(j in 1:length(condensates)){
            filtered_df = df %>%
                filter(mRNA == genes[i], Condensate %in% c(condensates[j], "PBS"))

            # changing contrasts
            contrasts(filtered_df$Burn_Condition) = cbind(c(-1,1,0), c(-1,0,1)) # meaning is specified above
            # ANCOVA
            ancova = aov(Scaled_Intensity ~ Sex + Burn_Condition, data = filtered_df)

            # Tukey's post hoc test
            tukeys_ancova = summary(glht(ancova, linfct = mcp(Burn_Condition = "Tukey")), test = adjusted("none"))
            tukeys_test = tukeys_ancova$test
            values_df = data.frame("ANCOVA", condensates[j], genes[i], conc, time, tukeys_test$tstat, 
                                   tukeys_test$pvalues) %>%
                rownames_to_column(var = "Comparison")

            # saving values
            tukey_df = rbind(tukey_df, values_df)
        }
    }
    
    # changing col names
    colnames(tukey_df)[2:8] = c("Model", "Condensate", "mRNA", "Concentration", "Time Point", "Statistic", 
                                "P Value")
    
 
    # calculating padj values
    PAdj = c()
    for(j in 1:length(condensates)){
        filtered_df = tukey_df %>%
            filter(Condensate == condensates[j])
        padj = p.adjust(as.numeric(as.character(filtered_df$`P Value`)), method = "fdr")
        PAdj = c(PAdj, padj)

    }
    
    tukey_df$`P Adj` = PAdj
    
    return(tukey_df)
}

# calling fn
mRNA_conc1_24_ancova_tukey = get_ancova(conc1_24_mRNA_df)
mRNA_conc1_72_ancova_tukey = get_ancova(conc1_72_mRNA_df)
mRNA_conc25_4_ancova_tukey = get_ancova(conc25_4_mRNA_df)
mRNA_conc25_24_ancova_tukey = get_ancova(conc25_24_mRNA_df)
mRNA_conc25_72_ancova_tukey = get_ancova(conc25_72_mRNA_df)
mRNA_conc5_24_ancova_tukey = get_ancova(conc5_24_mRNA_df)
mRNA_conc5_72_ancova_tukey = get_ancova(conc5_72_mRNA_df)

In [9]:
# creating 1 df 
anova_ancova_df = rbind(mRNA_conc1_24_anova_tukey, mRNA_conc1_72_anova_tukey, mRNA_conc25_4_anova_tukey,
                       mRNA_conc25_24_anova_tukey, mRNA_conc25_72_anova_tukey, mRNA_conc5_24_anova_tukey,
                       mRNA_conc5_72_anova_tukey, mRNA_conc1_24_ancova_tukey, mRNA_conc1_72_ancova_tukey, 
                        mRNA_conc25_4_ancova_tukey, mRNA_conc25_24_ancova_tukey, mRNA_conc25_72_ancova_tukey, 
                        mRNA_conc5_24_ancova_tukey, mRNA_conc5_72_ancova_tukey) %>%
    # changing condensates so they're more legible
    mutate(Condensate = ifelse(Condensate == "C", "Cardboard",
                               ifelse(Condensate == "P", "Plastic", "PBS")))
head(anova_ancova_df)

,Comparison,Model,Condensate,mRNA,Concentration,Time Point,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,S - PBS,ANOVA,Cardboard,HMOX1,1,24,2.564202,0.02158131,0.6150673
2,F - PBS,ANOVA,Cardboard,HMOX1,1,24,1.690263,0.11164628,0.6960601
3,F - S,ANOVA,Cardboard,HMOX1,1,24,-0.873939,0.39592818,0.6960601
4,S - PBS,ANOVA,Plastic,HMOX1,1,24,2.528121,0.02318106,0.6960601
5,F - PBS,ANOVA,Plastic,HMOX1,1,24,1.735654,0.10311520,0.8192518
6,F - S,ANOVA,Plastic,HMOX1,1,24,-0.792467,0.44044264,0.6960601


In [10]:
# exporting
write.csv(anova_ancova_df, paste0(Output,"/", cur_date, "_mRNA_ANOVA_ANCOVA.csv"), row.names = FALSE)